# Statistical Analysis: Genius Song Lyrics Subset (1%)

**Dataset:** 51'349 Songs | 5'333 Artists | 6 Genres

**Purpose:** Explore patterns and distributions in song lyrics across genres and artists. Focus on word frequencies, stylistic differences and similarity structures.

---

# 1. Dataset Overview
## 1.1 Import Packages and Settings

In [ ]:
import pandas as pd
import re
from collections import defaultdict, Counter
from itertools import tee
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LinearRegression
import os

In [ ]:
plt.style.use('default')
%matplotlib inline

## 1.2 Load Dataset

In [ ]:
df = pd.read_csv('data/clean/lyrics_subset_1pct_clean.csv')
df = df[df["language_cld3"] == "en"] # only english songs, better for analysis

print(f"DataFrame shape: {df.shape}")
print(f"Number of Songs: {len(df)} | Artists: {df['artist'].nunique()}")
df.head()

## 1.3 Descriptive Statistics

In [ ]:
# Genre distribution
print("\nGENRE DISTRIBUTION")
print("=" * 60)
category_counts = df['tag'].value_counts().sort_values(ascending=False)

for tag,count in category_counts.items():
    pct = (count / len(df)) * 100
    print(f"{tag}: {count:,} songs ({pct:.2f}%)")

In [ ]:
category_counts.plot(kind="bar")
plt.title("Genre Distribution", fontweight='bold')
plt.show()

# 2. Tokenization
## 2.1 Build Token

In [ ]:
def preprocess_text(text, lowercase=True):
    """Clean and tokenize text"""
    if not isinstance(text, str):
        return []
    if lowercase:
        text = text.lower()

    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    return tokens

preprocess_text("This is a test!")

In [ ]:
df["tokens_raw"] = df["lyrics"].apply(preprocess_text)
df["word_count"] = df["tokens_raw"].apply(len) # words per song

df[['title','artist','word_count']].head()

In [ ]:
total_lyrics = len(df)
total_words_raw = df["word_count"].sum()
avg_words_raw = df["word_count"].mean()
min_words_raw = df["word_count"].min()
max_words_raw = df["word_count"].max()

print("TEXT STATISTICS")
print("=" * 60)
print(f"Total lyrics (songs):     {total_lyrics:,}")
print(f"Total words:              {total_words_raw:,}")
print(f"Average words/lyric:      {avg_words_raw:.2f}")
print(f"Shortest lyric:           {min_words_raw} words")
print(f"Longest lyric:            {max_words_raw} words")

## 2.2 Filter Stopwords

In [ ]:
STOPWORDS = {
    "the","a","an","and","or","but","if","then","so","than","that","those","these","this",
    "to","of","in","on","for","with","as","at","by","from","into","over","under","up","down",
    "is","am","are","was","were","be","been","being","do","does","did","doing","have","has","had",
    "i","you","he","she","it","we","they","me","him","her","us","them","my","your","his","its","our","their",
    "not","no","yes","yeah","y'all","yall","im","i'm","i’d","i'd","i’ll","i'll","youre","you're","dont","don't",
    "cant","can't","ill","i’ll","id","i'd","ive","i’ve","ya","oh","ooh","la","na","nah"
}

def filtered_tokens(text):
    """filter stopwords"""
    tokens = preprocess_text(text)
    return [t for t in tokens if t not in STOPWORDS and not t.isdigit() and len(t) > 1]

filtered_tokens("This is a test!")

In [ ]:
df["tokens"] = df["lyrics"].apply(filtered_tokens)
tokens_per_row = df["tokens"]
tokens = [t for row in tokens_per_row for t in row]
df["token_count"] = df["tokens"].apply(len)

df[['title','artist','word_count','token_count']].head()

In [ ]:
total_lyrics = len(df)
total_tokens = len(tokens)
unique_tokens = len(set(tokens))
avg_tokens = df["token_count"].mean()
min_tokens = df["token_count"].min()
max_tokens = df["token_count"].max()

print("TOKEN STATISTICS")
print("=" * 60)
print(f"Total lyrics (songs):     {total_lyrics:,}")
print(f"Total tokens:             {total_tokens:,}")
print(f"Unique tokens:            {unique_tokens:,}")
print(f"Average tokens/lyric:     {avg_tokens:.2f}")
print(f"Shortest lyric:           {min_tokens} tokens")
print(f"Longest lyric:            {max_tokens} tokens")

In [ ]:
top_n = 15
tokens_raw = [t for row in df["tokens_raw"] for t in row]
tokens_filtered = [t for row in df["tokens"] for t in row]

word_counts_raw = Counter(tokens_raw).most_common(top_n)
word_counts_filtered = Counter(tokens_filtered).most_common(top_n)

df_raw = pd.DataFrame(word_counts_raw, columns=["word", "count"])
df_filtered = pd.DataFrame(word_counts_filtered, columns=["word", "count"])


fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

axes[0].bar(df_raw["word"], df_raw["count"])
axes[0].set_title(f"Top {top_n} Words (Before Stopword Removal)", fontsize=12, fontweight='bold')
axes[0].set_ylabel("Frequency")
axes[0].tick_params(axis="x", rotation=45)

axes[1].bar(df_filtered["word"], df_filtered["count"])
axes[1].set_title(f"Top {top_n} Words (After Stopword Removal)", fontsize=12, fontweight='bold')
axes[1].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

---
# 3. Word-Level Analysis
## 3.1 Vocabulary Statistics

In [ ]:
# get all words
all_tokens = [token for tokens in df["tokens_raw"] for token in tokens]

# count unique words
word_counts = Counter(all_tokens)
vocab_size = len(word_counts)
type_token_ratio = vocab_size / len(all_tokens)

print("VOCABULARY STATISTICS")
print("=" * 60)
print(f"Total word tokens:          {len(all_tokens):,}")
print(f"Unique words (vocabulary):  {vocab_size:,}")
print(f"Type-token ratio:           {type_token_ratio:.4f}")

On average, each word appears roughly 100 times in the dataset, indicating a high level of repetition.

The type–token ratio (TTR) of 0.012 is relatively low, which was to be expected since the corpus consists of song lyrics — a genre characterized by recurring words, refrains, and limited lexical variety compared to other types of text.

## 3.2 Zipf's Law Analysis

**Zipf's Law:** In natural language, word frequency is inversely proportional to rank.

Mathematical form: **f(r) = C / r^α**

Where:
- f(r) = frequency at rank r
- α = slope (ideal = -1.0 for natural language)
- C = coefficient

If α = -1.0, then rank 2 word appears half as often as rank 1, rank 3 appears 1/3 as often, etc.

In [ ]:
all_word_freq = Counter(tokens_raw).most_common(100)
ranks = list(range(1, len(all_word_freq) + 1))
frequencies = [freq for word, freq in all_word_freq]

# Fit power law model (top 100 words)
log_ranks_100 = np.log(ranks).reshape(-1, 1)
log_freq_100 = np.log(frequencies)

model = LinearRegression()
model.fit(log_ranks_100, log_freq_100)

r_squared = model.score(log_ranks_100, log_freq_100)
slope = model.coef_[0]
intercept = model.intercept_
coefficient_C = np.exp(intercept)

print("ZIPF'S LAW ANALYSIS")
print("=" * 60)
print(f"Fitted equation: f(r) = {coefficient_C:.2f} / r^{abs(slope):.3f}")
print(f"\nModel parameters:")
print(f"  Slope (α):         {slope:.4f}")
print(f"  R^2 (fit quality): {r_squared:.4f}")
print(f"  Ideal Zipf slope:  -1.0000")
print(f"  Deviation:         {abs(slope + 1.0):.4f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Log-log plot
axes[0].loglog(ranks, frequencies, 'o', alpha=0.7, color='steelblue', label='Actual')
fitted_zipf = [coefficient_C / (r ** abs(slope)) for r in ranks]
axes[0].loglog(ranks, fitted_zipf, 'g-', linewidth=2, alpha=0.8, label=f'Fitted (α={abs(slope):.3f})')
ideal_zipf = [frequencies[0] / r for r in ranks]
axes[0].loglog(ranks, ideal_zipf, 'r--', linewidth=2, alpha=0.7, label='Ideal Zipf (α=1.0)')
axes[0].set_xlabel('Rank (log scale)')
axes[0].set_ylabel('Frequency (log scale)')
axes[0].set_title("Zipf's Law: Actual vs Fitted vs Ideal", fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Top 30 comparison
n = 30
axes[1].plot(ranks[:n], frequencies[:n], 'o', color='steelblue', label='Actual')
axes[1].plot(ranks[:n], [coefficient_C / (r ** abs(slope)) for r in ranks[:n]],
             'g-', label=f'Fitted (α={abs(slope):.3f})', linewidth=2, alpha=0.7)
axes[1].plot(ranks[:n], [frequencies[0] / r for r in ranks[:n]],
             'r--', label='Ideal Zipf (a=1.0)', linewidth=2, alpha=0.6)
axes[1].set_xlabel('Rank')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Top 30 Words: Detailed Comparison', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

The fitted Zipf’s Law model shows a slope of −0.83 with an $R^2$ of 0.98, indicating an excellent fit to the expected distribution.

Although the slope is slightly flatter than the ideal −1.0, this small deviation (0.17) suggests that the frequency–rank relationship in the lyrics follows Zipf’s Law closely — common words are used much more frequently than rare ones, as typically observed in song lyrics.

## 3.3 Hapax Legomena (Rare Words)

**Hapax legomena** = words appearing only once in the corpus

These are the rarest words. High hapax count suggests diverse vocabulary.

In [ ]:
# Find hapax legomena
word_counts = Counter(tokens_raw)
hapax = [word for word, count in word_counts.items() if count == 1]
hapax_pct = (len(hapax) / vocab_size) * 100

# Find words appearing 2-5 times
rare_2 = [word for word, count in word_counts.items() if count == 2]
rare_3_5 = [word for word, count in word_counts.items() if 3 <= count <= 5]
rare_le_5 = len(hapax) + len(rare_2) + len(rare_3_5)

print("RARE WORDS ANALYSIS")
print("=" * 60)
print(f"Hapax legomena (count=1):     {len(hapax):,} words ({hapax_pct:.1f}% of vocab)")
print(f"Words appearing twice:        {len(rare_2):,} words")
print(f"Words appearing 3-5 times:    {len(rare_3_5):,} words")
print(f"\nTotal rare words (≤5 times):  {rare_le_5:,} words ({(rare_le_5 / vocab_size) * 100:.1f}% of vocab)")
print(f"\nExamples of hapax legomena:")
print(f"  {hapax[:10]}")

A large proportion of the vocabulary in the lyrics is rare: 48.6% are hapax legomena (appearing only once), and 74.3% of all words occur five times or fewer. A few words are repeated frequently, while most words are unique or very infrequent.

In [ ]:
freq_distribution = Counter(word_counts.values())
freq_bins = sorted(freq_distribution.keys())[:20]  # First 20 bins
freq_counts = [freq_distribution[f] for f in freq_bins]

plt.figure()
plt.bar(freq_bins, freq_counts, color='steelblue', edgecolor='black')
plt.xlabel('Word Frequency')
plt.ylabel('Number of Words')
plt.title('Distribution: How Many Words Appear X Times?', fontweight='bold')
plt.xticks(freq_bins)
plt.tight_layout()
plt.show()

## 3.4 Category Statistics

In [ ]:
categories = df['tag'].unique()

category_stats = {}
for cat in categories:
    cat_df = df[df['tag'] == cat]
    cat_text = ' '.join(cat_df['lyrics'].str.lower())
    cat_words = cat_text.split()
    cat_vocab = len(set(cat_words))

    has_number_pct = sum(any(char.isdigit() for char in lyric) for lyric in cat_df['lyrics']) / len(cat_df) * 100

    category_stats[cat] = {
        'songs': len(cat_df),
        'total_words': len(cat_words),
        'avg_words': len(cat_words) / len(cat_df),
        'vocab': cat_vocab,
        'has_number_pct': has_number_pct
    }

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

avg_words_cat = [category_stats[cat]['avg_words'] for cat in categories]
axes[0].bar(categories, avg_words_cat, color='steelblue')
axes[0].set_title('Average Words per Song', fontweight='bold')
axes[0].set_ylabel('Words')
axes[0].set_xticklabels(categories, rotation=45)

vocab_sizes = [category_stats[cat]['vocab'] for cat in categories]
axes[1].bar(categories, vocab_sizes, color='coral')
axes[1].set_title('Vocabulary Size per Genre', fontweight='bold')
axes[1].set_ylabel('Unique Words')
axes[1].set_xticklabels(categories, rotation=45)

has_number_pct = [category_stats[cat]['has_number_pct'] for cat in categories]
axes[2].bar(categories, has_number_pct, color='lightgreen')
axes[2].set_title('Songs Containing Numbers (%)', fontweight='bold')
axes[2].set_ylabel('Percentage')
axes[2].set_xticklabels(categories, rotation=45)

plt.tight_layout()
plt.show()

---
# 4. N-gram Analysis
## 4.1 Unigram, Bigram, Trigram

In [ ]:
def ngrams(tokens, n):
    """generate n-grams"""
    if n <= 0:
        return []
    iters = tee(tokens, n)
    for i, it in enumerate(iters):
        for _ in range(i):
            next(it, None)
    return zip(*iters)

In [ ]:
unigram_counts = Counter()
bigram_counts = Counter()
trigram_counts = Counter()

for tokens in df["tokens"]:
    unigram_counts.update(tokens)
    bigram_counts.update(ngrams(tokens, 2))
    trigram_counts.update(ngrams(tokens, 3))

top_unigrams = pd.DataFrame(unigram_counts.most_common(15), columns=["word", "count"])

top_bigrams = pd.DataFrame(
    [(" ".join(k), v) for k, v in bigram_counts.most_common(15)],
    columns=["bigram", "count"]
)
top_trigrams = pd.DataFrame(
    [(" ".join(k), v) for k, v in trigram_counts.most_common(15)],
    columns=["trigram", "count"]
)

In [ ]:
print("TOP 15 UNIGRAMS:")
print("=" * 60)
print(top_unigrams)

In [ ]:
print("TOP 15 BIGRAMS:")
print("=" * 60)
print(top_bigrams)

In [ ]:
print("TOP 15 TRIGRAMS:")
print("=" * 60)
print(top_trigrams)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].barh(top_unigrams["word"][::-1], top_unigrams["count"][::-1])
axes[0].set_title("Top 15 Unigrams", fontweight='bold')
axes[0].set_xlabel("Frequency")

axes[1].barh(top_bigrams["bigram"][::-1], top_bigrams["count"][::-1])
axes[1].set_title("Top 15 Bigrams", fontweight='bold')
axes[1].set_xlabel("Frequency")

axes[2].barh(top_trigrams["trigram"][::-1], top_trigrams["count"][::-1])
axes[2].set_title("Top 15 Trigrams", fontweight='bold')
axes[2].set_xlabel("Frequency")

plt.tight_layout()
plt.show()

## 4.2 N-Grams per Artist/Genre

In [ ]:
def most_common_ngram_for_group(group_df: pd.DataFrame, label_col: str, n: int) -> pd.DataFrame:
    """
    returns, for each group (artist/tag), the most frequent n-gram along with its count.
    Columns: [label_col, 'ngram', 'count', 'songs']
    """
    rows = []
    for label, sub in group_df.groupby(label_col):
        c = Counter()
        for toks in sub["tokens"]:
            c.update(ngrams(toks, n))
        if c:
            top_ngram, cnt = c.most_common(1)[0]
            rows.append({label_col: label, "ngram": " ".join(top_ngram), "count": cnt, "songs": len(sub)})
        else:
            rows.append({label_col: label, "ngram": None, "count": 0, "songs": len(sub)})
    return pd.DataFrame(rows).sort_values([label_col]).reset_index(drop=True)

In [ ]:
top_unigrams_artist = most_common_ngram_for_group(df, "artist", n=1).sort_values("count", ascending=False).head(20)
top_bigrams_artist  = most_common_ngram_for_group(df, "artist", n=2).sort_values("count", ascending=False).head(20)
top_trigrams_artist = most_common_ngram_for_group(df, "artist", n=3).sort_values("count", ascending=False).head(20)

In [ ]:
# create labels
top_unigrams_artist["label"] = top_unigrams_artist["artist"] + " - " + top_unigrams_artist["ngram"]
top_bigrams_artist["label"]  = top_bigrams_artist["artist"]  + " - " + top_bigrams_artist["ngram"]
top_trigrams_artist["label"] = top_trigrams_artist["artist"] + " - " + top_trigrams_artist["ngram"]


fig, axes = plt.subplots(1, 3, figsize=(18, 7))

axes[0].barh(top_unigrams_artist["label"][::-1], top_unigrams_artist["count"][::-1])
axes[0].set_title("Top 20 Unigrams per Artist", fontweight='bold')
axes[0].set_xlabel("Frequency")

axes[1].barh(top_bigrams_artist["label"][::-1], top_bigrams_artist["count"][::-1])
axes[1].set_title("Top 20 Bigrams per Artist", fontweight='bold')
axes[1].set_xlabel("Frequency")

axes[2].barh(top_trigrams_artist["label"][::-1], top_trigrams_artist["count"][::-1])
axes[2].set_title("Top 20 Trigrams per Artist", fontweight='bold')
axes[2].set_xlabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
top_unigrams_genre = most_common_ngram_for_group(df, "tag", n=1).sort_values("count", ascending=False)
top_bigrams_genre  = most_common_ngram_for_group(df, "tag", n=2).sort_values("count", ascending=False)
top_trigrams_genre = most_common_ngram_for_group(df, "tag", n=3).sort_values("count", ascending=False)

In [ ]:
# create labels
top_unigrams_genre["label"] = top_unigrams_genre["tag"] + " - " + top_unigrams_genre["ngram"]
top_bigrams_genre["label"]  = top_bigrams_genre["tag"]  + " - " + top_bigrams_genre["ngram"]
top_trigrams_genre["label"] = top_trigrams_genre["tag"] + " - " + top_trigrams_genre["ngram"]


fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].barh(top_unigrams_genre["label"][::-1], top_unigrams_genre["count"][::-1])
axes[0].set_title("Top Unigrams per Genre", fontweight='bold')
axes[0].set_xlabel("Frequency")

axes[1].barh(top_bigrams_genre["label"][::-1], top_bigrams_genre["count"][::-1])
axes[1].set_title("Top Bigrams per Genre", fontweight='bold')
axes[1].set_xlabel("Frequency")

axes[2].barh(top_trigrams_genre["label"][::-1], top_trigrams_genre["count"][::-1])
axes[2].set_title("Top Trigrams per Genre", fontweight='bold')
axes[2].set_xlabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
df.head()

In [ ]:
output_dir = "data/clean"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "data.csv")

df = df.rename(columns={'tokens_raw': 'words'})

df.to_csv(output_path, index=False)

print(f"Cleaned Subset saved to: {output_path}")